In [1]:
!pip install jieba

In [2]:
!pip install gensim

In [3]:
!pip install plotly

     |████████████████████████████████| 21.8 MB 20.9 MB/s eta 0:00:01


In [4]:
!pip install requests

In [5]:
!pip install feedparser

     |████████████████████████████████| 81 kB 12.9 MB/s eta 0:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6064 sha256=87072e5983ee284064011a2b67a94379278bf89e745914ae4804769adcabb0a0
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [6]:
!pip install lxml

     |████████████████████████████████| 6.3 MB 20.5 MB/s eta 0:00:01


In [7]:
document = "科技大廠蘋果本週爆出重啟電動車計畫，據稱5年內上市、搭配突破性降低成本、提升續航里程的電池技術，引發市場話題"

In [8]:
text_corpus = [
    "兩項最新研究指出，具有2019冠狀病毒疾病（COVID-19）抗體的民眾，再度染疫的可能性相當低。研究人員還發現，抗體保護力能維持6個月，甚至是更久時間。",
    "由俄國富豪尤里·米爾納注資 1 億美元、已故英國天文物理學家霍金啟動的突破聆聽（Breakthrough Listen）計畫，近日收到了一個有趣的無線電訊號，來自距離太陽最近的另一顆恆星比鄰星。",
    "科技大廠蘋果本週爆出重啟電動車計畫，據稱5年內上市、搭配突破性降低成本、提升續航里程的電池技術，引發市場話題",
    "中國監管單位對阿里巴巴集團展開反壟斷調查，使阿里巴巴美國存託憑證（ADR）在週四（24日）創下有史以來最大跌幅。",
    "日本就有一名高齡95歲的阿嬤相當喜歡用GB遊戲主機玩「俄羅斯方塊」，無奈電子產品終究無法抵抗時間的摧殘而損壞，沒想到阿嬤親筆寫信給任天堂官方後，竟然得到一台全新的遊戲主機，讓網友感動不已。"
]

In [9]:
len(text_corpus)

5

### 準備資料

In [10]:
import feedparser

In [11]:
rss_url = "https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money"

In [12]:
newsFeed = feedparser.parse(rss_url)

In [13]:
i = 1
for e in newsFeed['entries']:
    title = e['title']
    link_url = e['links'][0]['href']
    print("%s, %s, %s"%(i, title, link_url))
    i = i + 1

1, 網安躍升拜登首要議題  攜手科技巨擘研擬新規範, https://money.udn.com/money/story/10511/5701435
2, 貴森森總部逃 胡佛報告：企業速離加州前進德州, https://money.udn.com/money/story/10511/5701200
3, 美眾院查國會之亂 首向政府索川普親信通聯等資料, https://money.udn.com/money/story/10511/5701088
4, 世衛國際專家組：疫情溯源機會之窗正變小, https://money.udn.com/money/story/10511/5701086
5, 阿富汗的眼淚（下）／台商眼中的阿富汗零商機 大陸有機會取得當地富礦？, https://money.udn.com/money/story/10511/5700568
6, 阿富汗的眼淚（上）／美國急撤阿富汗，最想推大陸入坑？, https://money.udn.com/money/story/10511/5700287
7, 彭博：華爾街和北京政府官員將恢復對話 以謀求共識, https://money.udn.com/money/story/10511/5698328
8, 美國准買汽車晶片！華為業務轉向有宏大企圖, https://money.udn.com/money/story/10511/5698185
9, 美疫情調查報告到期 真相恐難水落石出, https://money.udn.com/money/story/10511/5697815
10, 搞砸阿富汗撤軍 拜登最新民調支持度大失血, https://money.udn.com/money/story/10511/5697681
11, 斷芯周年前夕 美鬆綁向華為出售汽車晶片, https://money.udn.com/money/story/10511/5697630
12, 美疫源調查報告 中國是否為源頭恐無明確解答, https://money.udn.com/money/story/10511/5697063
13, 紐約州長古莫陷性騷醜聞 臨別演說怪罪媒體炒作, https://money.udn.com/money/story/10511/5694786
14, 美議員致函立陶宛駐美大使

In [14]:
import requests
from lxml import etree
from io import StringIO
import jieba
import pandas as pd
import time
import random
import pprint

each_article_text_list = []
news_crawl_result_file = 'news_crawl_result.txt'

In [15]:
#設定 request header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
my_headers = {
    'User-Agent': user_agent,
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "cache-control": "no-cache",
    "Accept-Charset": "UTF8,utf-8;q=0.7,*;q=0.7"
}

In [16]:
#爬文
for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url, headers = my_headers)
    if r.status_code == 200:
        parse_tree = etree.parse(StringIO(r.text), etree.HTMLParser())
        article_elements = parse_tree.xpath('//*[@id="article_body"]//p')
        post = []
        for a_part in article_elements:
            if type(a_part.text) is str:
                #each_article_text_list.append(a_part.text.strip())
                post.append(a_part.text.strip())
        each_article_text_list.append(post)
        #print(each_article_text_list)
        sleep_time = random.randint(3,10)
        print("sleep time: %s sec"%(sleep_time))
        time.sleep(sleep_time)
    #all_article_text = ''.join(each_article_text_list)
#all_article_text

https://money.udn.com/money/story/10511/5701435
sleep time: 8 sec
https://money.udn.com/money/story/10511/5701200
sleep time: 6 sec
https://money.udn.com/money/story/10511/5701088
sleep time: 6 sec
https://money.udn.com/money/story/10511/5701086
sleep time: 5 sec
https://money.udn.com/money/story/10511/5700568
sleep time: 9 sec
https://money.udn.com/money/story/10511/5700287
sleep time: 4 sec
https://money.udn.com/money/story/10511/5698328
sleep time: 3 sec
https://money.udn.com/money/story/10511/5698185
sleep time: 4 sec
https://money.udn.com/money/story/10511/5697815
sleep time: 7 sec
https://money.udn.com/money/story/10511/5697681
sleep time: 3 sec
https://money.udn.com/money/story/10511/5697630
sleep time: 3 sec
https://money.udn.com/money/story/10511/5697063
sleep time: 8 sec
https://money.udn.com/money/story/10511/5694786
sleep time: 4 sec
https://money.udn.com/money/story/10511/5694779
sleep time: 6 sec
https://money.udn.com/money/story/10511/5692463
sleep time: 10 sec
https://m

In [17]:
# 存檔
with open(news_crawl_result_file, 'wb') as saveFile:
    for post in each_article_text_list:
        if len(post) > 0:
            for line in post:
                line = line.encode('utf-8')
                saveFile.write(line)
                saveFile.write('\n'.encode())

In [18]:
import jieba
import time
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [19]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O ../jieba_data/dict.txt.big

--2021-08-26 21:05:37--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2021-08-26 21:05:38--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘../jieba_data/dict.txt.big’

../jieba_data/dict. 100%[===================>]   8.18M  --.-KB/s    in 0.04s   

2021-08-26 21:05:38 (229 MB/s) - ‘../jieba_data/dict.txt.big’ saved [8583143/8583143]



In [20]:
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt -O ../jieba_data/stop_words.txt

--2021-08-26 21:05:39--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222 [text/plain]
Saving to: ‘../jieba_data/stop_words.txt’

../jieba_data/stop_ 100%[===================>]     222  --.-KB/s    in 0s      

2021-08-26 21:05:39 (11.2 MB/s) - ‘../jieba_data/stop_words.txt’ saved [222/222]



In [21]:
jieba.set_dictionary('../jieba_data/dict.txt.big')

In [22]:
jieba.load_userdict('../jieba_data/mydict.txt')

Building prefix dict from /home/jovyan/work/jieba_data/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ud3c40470d3518910bfd0dc86d865fad4.cache
Loading model cost 1.944 seconds.
Prefix dict has been built successfully.


In [23]:
# 開檔後一次讀一行
stop_words_list = []
with open(file='../jieba_data/stop_words.txt',mode='r', encoding="UTF-8") as file:
    for line in file:
        line = line.strip()
        stop_words_list.append(line)
stop_words_list

['the',
 'of',
 'is',
 'and',
 'to',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '的',
 '了',
 '和',
 '是',
 '就',
 '都',
 '而',
 '及',
 '與',
 '著',
 '或',
 '一個',
 '沒有',
 '我們',
 '你們',
 '妳們',
 '他們',
 '她們',
 '是否']

### 分詞

In [24]:
seg_result = []
for line in text_corpus:
    seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [25]:
seg_result = []
for post in each_article_text_list:
    for line in post:
        seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [26]:
seg_stop_result = []
for line in seg_result:
    #print("[%s]"%(line))
    new_line = []
    for word in line.split():
        if word not in stop_words_list:
            #print(word)
            new_line.append(word)
    #print(new_line)
    seg_stop_result.append(' '.join(new_line))

In [27]:
'（' not in stop_words_list

True

In [28]:
'妳' not in stop_words_list

True

In [29]:
len(seg_stop_result)

194

In [30]:
seg_stop_result

['美國 總統 拜登 要求 民營 業者 應 加強 網路 安全 ， 美政府 今天 表示 ， 將和 業界 合作 研擬 出 一套 新 規範 ， 以 提升 科技 供應鏈 安全 。',
 '今天 白宮 一場 會議 ， 拜登 部會 官員 、 科技 巨擘 、 金融 業界 、 基礎設施 公司 等 主管 說 ， 愈來愈多 網路 攻擊 威脅 美國 經濟 ， 將 加強 因應 。',
 '拜登 在 白宮 東廳 （ East Room ） 對 戴 口罩 眾多 主管 表示 ： 「 聯邦政府 無法 單獨 應對 挑戰 ， 我 相信 必須 要 有 權力 、 能力 負責 任地 來 加強 網路 安全 。 」',
 '白宮 於 會後 表示 ， 美國 國家標準 技術 研究所 （ National Institute Standards Technology ， NIST ） 將與 業界 及其 他 夥伴 共同 合作 研擬 新 規範 ， 以 建立 安全 技術 ， 並 評估 技術 安全性 ， 包括 開放 原始 碼 軟體 。',
 '路透社 報導 ， 微軟 （ Microsoft ） 、 谷歌 （ Google ） 、 美國 產業 保險公司 旅行家 集團 （ Travelers ） 資安 保險業 者 Coalition 等 承諾 參加 國家標準 技術 研究所 領導 新計畫 。',
 '在 一連串 深受 矚目 網攻 後 ， 包括 美國 資訊科技 公司 SolarWinds 、 燃油 管線 營運 業者 殖民 管線 （ ColonialPipeline ） 、 肉品 加工 大廠 JBS 美國 軟體 公司 Kaseya 等 遭受 攻擊 ， 資安 問題 成為 拜登 政府 最 重要 議題 。',
 '微軟 說 ， 未來 5 年內 將 投入 200 億美元 以 加強 網路 安全 ， 這個 金額 現行 投資 4 倍 ； 此外 ， 將在 技術 服務 方面 投入 1.5 億美元 ， 以 協助 聯邦 、 州 地方 政府 維護 更新 安全 系統 。',
 'IBM 公司 則說 ， 將在 3 年內 培訓 超過 15 萬名 網路 安全 技術 人才 ， 並與 大專院校 合作 ， 以 培育 多元化 資安 工作人員 。',
 'Google 表示 ， 將在 5 年內 投資 100 億美元 ， 但 未 立即 說明 要 在 哪些項目 支出 。',
 '

In [31]:
#製作 sentence 的時候的樣子
for line in seg_stop_result:
    print(line.split())

['美國', '總統', '拜登', '要求', '民營', '業者', '應', '加強', '網路', '安全', '，', '美政府', '今天', '表示', '，', '將和', '業界', '合作', '研擬', '出', '一套', '新', '規範', '，', '以', '提升', '科技', '供應鏈', '安全', '。']
['今天', '白宮', '一場', '會議', '，', '拜登', '部會', '官員', '、', '科技', '巨擘', '、', '金融', '業界', '、', '基礎設施', '公司', '等', '主管', '說', '，', '愈來愈多', '網路', '攻擊', '威脅', '美國', '經濟', '，', '將', '加強', '因應', '。']
['拜登', '在', '白宮', '東廳', '（', 'East', 'Room', '）', '對', '戴', '口罩', '眾多', '主管', '表示', '：', '「', '聯邦政府', '無法', '單獨', '應對', '挑戰', '，', '我', '相信', '必須', '要', '有', '權力', '、', '能力', '負責', '任地', '來', '加強', '網路', '安全', '。', '」']
['白宮', '於', '會後', '表示', '，', '美國', '國家標準', '技術', '研究所', '（', 'National', 'Institute', 'Standards', 'Technology', '，', 'NIST', '）', '將與', '業界', '及其', '他', '夥伴', '共同', '合作', '研擬', '新', '規範', '，', '以', '建立', '安全', '技術', '，', '並', '評估', '技術', '安全性', '，', '包括', '開放', '原始', '碼', '軟體', '。']
['路透社', '報導', '，', '微軟', '（', 'Microsoft', '）', '、', '谷歌', '（', 'Google', '）', '、', '美國', '產業', '保險公司', '旅行家', '集團', '（', 'Travelers'

### word2vec 建模

In [32]:
from gensim.models import word2vec

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [33]:
class MySentence:
    def __init__(self, seg_result_list):
        self.seg_result_list = seg_result_list
        
    def __iter__(self):
        for line in self.seg_result_list:
            yield line.split()

In [34]:
sentence = MySentence(seg_stop_result)

In [35]:
start_time = time.time()
model = word2vec.Word2Vec( sentence , vector_size=300, sg=1, window=5, min_count=1, workers=cores-1)
print("--- spend %s seconds ---" % (time.time() - start_time))

--- spend 0.37775635719299316 seconds ---


### 視覺化

In [36]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np 

In [37]:
len(model.wv.vectors)

2406

In [38]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

In [73]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

In [40]:
model.wv.similar_by_word('拜登')

[('，', 0.998995840549469),
 ('。', 0.9989018440246582),
 ('）', 0.9988296627998352),
 ('「', 0.9987443089485168),
 ('（', 0.9987117648124695),
 ('在', 0.9986963868141174),
 ('對', 0.9986749887466431),
 ('美國', 0.9986046552658081),
 ('中國', 0.9985399842262268),
 ('他', 0.9984443187713623)]

In [41]:
model.wv.similar_by_word('阿富汗')

[('，', 0.9986484050750732),
 ('）', 0.9985809326171875),
 ('。', 0.9984713792800903),
 ('（', 0.9983435869216919),
 ('「', 0.9983258843421936),
 ('」', 0.9983129501342773),
 ('對', 0.9982768893241882),
 ('在', 0.9982525110244751),
 ('中國', 0.998239278793335),
 ('報告', 0.9981963634490967)]

## 潛在語意分析 

In [42]:
len(seg_stop_result)

194

In [43]:
processed_corpus = []
for post in seg_stop_result:
    post_corpus = []
    for text in post.split():
        post_corpus.append(text)
    processed_corpus.append(post_corpus)

In [44]:
len(processed_corpus)

194

In [45]:
#一個句子一個 list, 全部的句子再用一個 list 包起來 
processed_corpus

[['美國',
  '總統',
  '拜登',
  '要求',
  '民營',
  '業者',
  '應',
  '加強',
  '網路',
  '安全',
  '，',
  '美政府',
  '今天',
  '表示',
  '，',
  '將和',
  '業界',
  '合作',
  '研擬',
  '出',
  '一套',
  '新',
  '規範',
  '，',
  '以',
  '提升',
  '科技',
  '供應鏈',
  '安全',
  '。'],
 ['今天',
  '白宮',
  '一場',
  '會議',
  '，',
  '拜登',
  '部會',
  '官員',
  '、',
  '科技',
  '巨擘',
  '、',
  '金融',
  '業界',
  '、',
  '基礎設施',
  '公司',
  '等',
  '主管',
  '說',
  '，',
  '愈來愈多',
  '網路',
  '攻擊',
  '威脅',
  '美國',
  '經濟',
  '，',
  '將',
  '加強',
  '因應',
  '。'],
 ['拜登',
  '在',
  '白宮',
  '東廳',
  '（',
  'East',
  'Room',
  '）',
  '對',
  '戴',
  '口罩',
  '眾多',
  '主管',
  '表示',
  '：',
  '「',
  '聯邦政府',
  '無法',
  '單獨',
  '應對',
  '挑戰',
  '，',
  '我',
  '相信',
  '必須',
  '要',
  '有',
  '權力',
  '、',
  '能力',
  '負責',
  '任地',
  '來',
  '加強',
  '網路',
  '安全',
  '。',
  '」'],
 ['白宮',
  '於',
  '會後',
  '表示',
  '，',
  '美國',
  '國家標準',
  '技術',
  '研究所',
  '（',
  'National',
  'Institute',
  'Standards',
  'Technology',
  '，',
  'NIST',
  '）',
  '將與',
  '業界',
  '及其',
  '他',
  '夥伴',
  '共同',
  '合作',

In [46]:
from gensim import corpora
#建立辭典
dictionary = corpora.Dictionary(processed_corpus, prune_at=100)
print(dictionary)

Dictionary(2406 unique tokens: ['。', '一套', '今天', '以', '供應鏈']...)


In [47]:
#pprint.pprint(dictionary.token2id)

### 每個()中的第一個數字對應於字典中token 的ID，第二個數字對應於此token的計數。

In [48]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus ]
#pprint.pprint(bow_corpus)

In [49]:
new_text = "拜登 阿富汗"
#new_text = "台灣"

In [50]:
new_vec = dictionary.doc2bow(new_text.split())
print(new_vec)

[(11, 1), (897, 1)]


## 建立 tf-idf 模型

In [51]:
from gensim import models

In [52]:
tfidf = models.TfidfModel(bow_corpus)

In [53]:
corpus_tfidf = tfidf[bow_corpus]

In [54]:
for i in corpus_tfidf:
    pprint.pprint(i)

[(0, 0.010461752489594354),
 (1, 0.2604338888675145),
 (2, 0.12655252307787335),
 (3, 0.14659802387083173),
 (4, 0.2604338888675145),
 (5, 0.20612037812819914),
 (6, 0.17185236818184219),
 (7, 0.1918978689748006),
 (8, 0.28377210350261967),
 (9, 0.2604338888675145),
 (10, 0.1918978689748006),
 (11, 0.09935917385400425),
 (12, 0.2604338888675145),
 (13, 0.15180686738888374),
 (14, 0.20612037812819914),
 (15, 0.22616587892115755),
 (16, 0.2604338888675145),
 (17, 0.22616587892115755),
 (18, 0.18086603381718871),
 (19, 0.16423142247549036),
 (20, 0.12336184908208668),
 (21, 0.06055235584611094),
 (22, 0.2604338888675145),
 (23, 0.0890938391357297),
 (24, 0.14659802387083173),
 (25, 0.20612037812819914),
 (26, 0.032332959504995686)]
[(0, 0.010901006068513668),
 (2, 0.13186603520105877),
 (6, 0.17906786748224296),
 (11, 0.10353092927999503),
 (14, 0.21477467518512794),
 (18, 0.18845998643059284),
 (19, 0.1711269440590034),
 (21, 0.06309474432680338),
 (26, 0.033690511042727904),
 (27, 0.269

[(0, 0.00832358994888616),
 (23, 0.07088492914312813),
 (26, 0.025724781485817826),
 (27, 0.06850031178054321),
 (41, 0.08745299855256441),
 (61, 0.09576466002827043),
 (81, 0.16399370080251408),
 (125, 0.09576466002827043),
 (277, 0.1034016198840874),
 (303, 0.13066596813181464),
 (428, 0.11288752199203832),
 (473, 0.11663646594884616),
 (523, 0.15267797388630983),
 (554, 0.1254136256385826),
 (555, 0.07475698474976568),
 (560, 0.10631659323664396),
 (564, 0.13672935255478683),
 (565, 0.11288752199203832),
 (566, 0.17994232213403705),
 (571, 0.10631659323664396),
 (580, 0.17994232213403705),
 (614, 0.17994232213403705),
 (620, 0.1254136256385826),
 (638, 0.2072066703817643),
 (639, 0.2072066703817643),
 (640, 0.2072066703817643),
 (641, 0.2072066703817643),
 (642, 0.2072066703817643),
 (643, 0.2072066703817643),
 (644, 0.1254136256385826),
 (645, 0.2072066703817643),
 (646, 0.2072066703817643),
 (647, 0.17994232213403705),
 (648, 0.17994232213403705),
 (649, 0.17994232213403705),
 (65

 (833, 0.13769470230298655),
 (834, 0.2527406066670711),
 (835, 0.2527406066670711),
 (836, 0.2527406066670711),
 (837, 0.21948488230348268),
 (838, 0.21948488230348268),
 (839, 0.2527406066670711),
 (840, 0.2527406066670711),
 (841, 0.2527406066670711),
 (842, 0.21948488230348268)]
[(0, 0.006230476035850893),
 (21, 0.03606183594849493),
 (26, 0.01925583018374621),
 (27, 0.05127469680965013),
 (41, 0.06546139527427933),
 (48, 0.03820408050691589),
 (49, 0.03820408050691589),
 (71, 0.10513939135265116),
 (72, 0.10513939135265116),
 (111, 0.12275458415840082),
 (158, 0.11428439691796884),
 (216, 0.12275458415840082),
 (253, 0.12275458415840082),
 (277, 0.07739945368670886),
 (339, 0.06840814001378599),
 (421, 0.1023463385055393),
 (437, 0.06409171227840404),
 (561, 0.1023463385055393),
 (562, 0.08449995793126557),
 (601, 0.12275458415840082),
 (625, 0.13469264257083036),
 (630, 0.10771440927494982),
 (657, 0.11428439691796884),
 (661, 0.1023463385055393),
 (743, 0.12275458415840082),
 (7

 (26, 0.021256760274748657),
 (54, 0.10366620155425234),
 (91, 0.18924003836095835),
 (264, 0.16947217246807442),
 (555, 0.09265917212347652),
 (568, 0.33232265044505577),
 (575, 0.16195677865197014),
 (596, 0.12479974087194108),
 (612, 0.22303342831076475),
 (686, 0.18924003836095835),
 (698, 0.16195677865197014),
 (985, 0.17836099671463143),
 (1009, 0.18924003836095835),
 (1269, 0.22303342831076475),
 (1288, 0.35672199342926286),
 (1323, 0.2568268182605712),
 (1324, 0.2568268182605712),
 (1325, 0.2568268182605712),
 (1326, 0.2568268182605712),
 (1327, 0.20326556241788082),
 (1328, 0.18924003836095835),
 (1329, 0.2568268182605712)]
[(0, 0.007266212684799999),
 (21, 0.1261699595496533),
 (26, 0.03742810788249906),
 (35, 0.10948185615315811),
 (48, 0.04455501839556647),
 (49, 0.04455501839556647),
 (79, 0.08789708506261393),
 (104, 0.15708356510254012),
 (197, 0.08163664144752698),
 (268, 0.13328271062784913),
 (327, 0.13328271062784913),
 (402, 0.15708356510254012),
 (437, 0.0747461365

 (11, 0.14376153440226816),
 (26, 0.03898512579753142),
 (42, 0.10605541508919138),
 (55, 0.07053095352103048),
 (80, 0.07188076720113408),
 (180, 0.07053095352103048),
 (390, 0.1308463905960893),
 (415, 0.1491164865904902),
 (421, 0.12432551108359229),
 (423, 0.1491164865904902),
 (568, 0.08126443958229346),
 (579, 0.1140363266045263),
 (766, 0.12432551108359229),
 (888, 0.1308463905960893),
 (897, 0.08707706194242751),
 (1047, 0.21964744011152068),
 (1441, 0.13882730211142422),
 (1444, 0.1491164865904902),
 (1467, 0.18840925312522008),
 (1468, 0.18840925312522008),
 (1469, 0.18840925312522008),
 (1470, 0.18840925312522008),
 (1471, 0.18840925312522008),
 (1472, 0.18840925312522008),
 (1473, 0.18840925312522008),
 (1474, 0.3272365552366443),
 (1475, 0.18840925312522008),
 (1476, 0.18840925312522008),
 (1477, 0.1491164865904902),
 (1478, 0.18840925312522008),
 (1479, 0.16361827761832215),
 (1480, 0.18840925312522008),
 (1481, 0.3272365552366443),
 (1482, 0.18840925312522008),
 (1483, 0

 (1851, 0.14707677274307174),
 (1852, 0.42720197610567967),
 (1869, 0.14707677274307174),
 (1870, 0.16936142653470154),
 (1871, 0.16936142653470154),
 (1872, 0.16936142653470154),
 (1873, 0.16936142653470154),
 (1874, 0.16936142653470154),
 (1875, 0.16936142653470154),
 (1876, 0.14707677274307174),
 (1877, 0.16936142653470154),
 (1878, 0.16936142653470154),
 (1879, 0.16936142653470154),
 (1880, 0.16936142653470154),
 (1881, 0.16936142653470154),
 (1882, 0.16936142653470154),
 (1883, 0.16936142653470154),
 (1884, 0.16936142653470154),
 (1885, 0.16936142653470154),
 (1886, 0.16936142653470154),
 (1887, 0.16936142653470154),
 (1888, 0.16936142653470154),
 (1889, 0.13404108593341477)]
[(0, 0.008373156062074998),
 (2, 0.10128743026890975),
 (8, 0.11355975545464582),
 (26, 0.02587796988634142),
 (33, 0.14475772859147193),
 (48, 0.05134258224440742),
 (49, 0.05134258224440742),
 (71, 0.09419810268126089),
 (72, 0.09419810268126089),
 (319, 0.14475772859147193),
 (494, 0.13754356164084036),
 (

 (819, 0.13634346812790854),
 (934, 0.1269356345062896),
 (946, 0.14960306489020841),
 (1085, 0.1269356345062896),
 (1271, 0.1042682041223708),
 (1458, 0.13634346812790854),
 (1543, 0.13634346812790854),
 (1561, 0.14960306489020841),
 (1584, 0.13634346812790854),
 (1617, 0.14960306489020841),
 (1652, 0.1196383518268027),
 (1830, 0.14960306489020841),
 (1980, 0.14960306489020841),
 (2007, 0.17227049527412722),
 (2008, 0.17227049527412722),
 (2009, 0.17227049527412722),
 (2010, 0.17227049527412722),
 (2011, 0.17227049527412722),
 (2012, 0.17227049527412722),
 (2013, 0.17227049527412722),
 (2014, 0.17227049527412722),
 (2015, 0.17227049527412722),
 (2016, 0.14960306489020841),
 (2017, 0.17227049527412722),
 (2018, 0.17227049527412722),
 (2019, 0.17227049527412722),
 (2020, 0.17227049527412722),
 (2021, 0.14960306489020841),
 (2022, 0.17227049527412722)]
[(0, 0.008937228096282713),
 (2, 0.1081108319119573),
 (20, 0.10538511446551743),
 (21, 0.15518534294390146),
 (26, 0.018414189172189264)

 (26, 0.015626032913403053),
 (33, 0.13111474980696036),
 (48, 0.0465036989107285),
 (49, 0.0465036989107285),
 (71, 0.04266010798013965),
 (72, 0.04266010798013965),
 (73, 0.10627292928558824),
 (76, 0.13111474980696036),
 (163, 0.1245804963062915),
 (407, 0.14942231682766363),
 (422, 0.1639538502823116),
 (448, 0.1639538502823116),
 (457, 0.13111474980696036),
 (463, 0.10627292928558824),
 (508, 0.1639538502823116),
 (674, 0.13111474980696036),
 (775, 0.14942231682766363),
 (1055, 0.14942231682766363),
 (1138, 0.13111474980696036),
 (1328, 0.27822405952187895),
 (1344, 0.1639538502823116),
 (1454, 0.1639538502823116),
 (1638, 0.14942231682766363),
 (1939, 0.1639538502823116),
 (2196, 0.14942231682766363),
 (2220, 0.1639538502823116),
 (2323, 0.3279077005646232),
 (2329, 0.14942231682766363),
 (2377, 0.18879567080368373),
 (2378, 0.18879567080368373),
 (2379, 0.18879567080368373),
 (2380, 0.18879567080368373),
 (2381, 0.18879567080368373),
 (2382, 0.18879567080368373),
 (2383, 0.18879

In [55]:
tfidf[new_vec]

[(11, 0.6366052632078169), (897, 0.7711898202492732)]

## 建立LSI 模型

In [56]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)

In [57]:
#LSI(潛在語意索引)
corpus_lsi = lsi[corpus_tfidf]

In [58]:
#目前的主題
lsi.print_topics()

[(0,
  '0.166*"、" + 0.164*"美國" + 0.160*"，" + 0.141*"（" + 0.141*"）" + 0.135*"在" + 0.126*"中國" + 0.126*"」" + 0.122*"「" + 0.118*"疫情"'),
 (1,
  '-0.325*"晶片" + -0.261*"華為" + -0.259*"汽車" + -0.173*"零部件" + -0.165*"出售" + -0.159*"用於" + -0.146*"申請" + -0.146*"批准" + -0.127*"引述" + -0.123*"供應商"'),
 (2,
  '0.224*"立陶宛" + 0.175*"台灣" + -0.137*"實驗室" + -0.131*"疫情" + -0.127*"世衛" + 0.117*"、" + -0.117*"國際" + 0.117*"阿富汗" + -0.112*"病毒" + -0.108*"溯源"'),
 (3,
  '-0.432*"立陶宛" + -0.297*"台灣" + -0.177*"中國" + -0.166*"克魯茲" + -0.165*"代表處" + -0.142*"報復" + -0.123*"大使" + -0.107*"致函" + -0.107*"設立" + -0.107*"讚賞"'),
 (4,
  '-0.254*"、" + -0.210*"加州" + -0.199*"德州" + -0.182*"企業" + 0.168*"疫苗" + 0.153*"接種" + 0.141*"他" + 0.138*"我" + -0.135*"總部" + 0.113*"」"'),
 (5,
  '0.174*"主席" + -0.173*"疫苗" + 0.159*"Fed" + -0.152*"接種" + 0.147*"官員" + -0.146*"我" + 0.137*"人士" + 0.133*"鮑爾" + 0.126*"拜登" + 0.119*"支持"'),
 (6,
  '0.219*"德州" + 0.191*"加州" + 0.146*"企業" + -0.134*"（" + -0.134*"）" + 0.129*"總部" + 0.123*"阿富汗" + 0.094*"國際" + 0.092*"結論" + 0.087*"有"'

In [59]:
#新文章
new_doc = '拜登近期因對撤離過程的處理而受到猛烈批評。最近幾天，撤離過程凸顯出喀布爾機場的混亂場面，人群拼命想離開這個國家。但拜登辯護，稱結束美國在阿富汗20年的參與不可避免的會出現問題。'
#new_vec_bow = dictionary.doc2bow()

In [60]:
#分詞
new_seg_result = jieba.lcut(new_doc)
len(new_seg_result)

50

In [61]:
new_doc_bow = dictionary.doc2bow(new_seg_result)

In [62]:
new_doc_lsi = lsi[new_doc_bow]

In [63]:
pprint.pprint(new_doc_lsi)

[(0, 1.876559821123212),
 (1, -0.015497871259731077),
 (2, 0.6480085026387342),
 (3, 0.5908338364923712),
 (4, 0.28404432248554107),
 (5, 0.71491652529305),
 (6, 0.20910630987922174),
 (7, -0.7953518556111125),
 (8, -0.33866230727018987),
 (9, -0.24121415783543654),
 (10, -0.2090170394440292),
 (11, -0.5083644768849782),
 (12, -0.27146230755936895),
 (13, -0.07811011957648129),
 (14, 0.15991135479332064),
 (15, -0.07874172803481505),
 (16, 0.3102007930947238),
 (17, -0.24589402588646425),
 (18, 0.09185089665521404),
 (19, 0.26166314658899714),
 (20, -0.27757275017530125),
 (21, -0.05300061989834526),
 (22, 0.16503637089384085),
 (23, 0.22309349630347014),
 (24, 0.03120287308076064),
 (25, 0.08737298481440559),
 (26, 0.19366779619559027),
 (27, 0.23183016979993865),
 (28, 0.17848197937580293),
 (29, -0.010695452135494467),
 (30, 0.03331971684726397),
 (31, 0.07781093933396767),
 (32, -0.026404135597088537),
 (33, -0.002316947562592382),
 (34, -0.046612072101978985),
 (35, -0.00387568426

### 建立索引

In [64]:
from gensim import similarities

In [65]:
index = similarities.MatrixSimilarity(corpus_lsi)

In [66]:
index

In [67]:
#新文章相似度
sims = index[new_doc_lsi]

In [68]:
sims

array([0.13267931, 0.13825005, 0.09365864, 0.06860705, 0.0379298 ,
       0.17806952, 0.08163874, 0.04305486, 0.10364569, 0.09177349,
       0.02787778, 0.03338594, 0.1699042 , 0.09172215, 0.04995544,
       0.08451286, 0.03662663, 0.03237831, 0.08052498, 0.04803829,
       0.03367383, 0.08886209, 0.03741225, 0.0282955 , 0.04028179,
       0.11728194, 0.07210511, 0.22418098, 0.16818322, 0.03496032,
       0.10242186, 0.05923358, 0.10520085, 0.10494021, 0.03748959,
       0.11697794, 0.10838953, 0.02646578, 0.06630291, 0.0426429 ,
       0.07198732, 0.        , 0.17453726, 0.        , 0.1622507 ,
       0.        , 0.2652105 , 0.        , 0.19594982, 0.14251162,
       0.        , 0.09604438, 0.        , 0.03336998, 0.        ,
       0.03113228, 0.        , 0.1366482 , 0.        , 0.2162612 ,
       0.05499911, 0.        , 0.0888596 , 0.        , 0.07552117,
       0.        , 0.04117655, 0.        , 0.1268207 , 0.        ,
       0.12248888, 0.        , 0.17600626, 0.        , 0.14105

In [69]:
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.13267931), (1, 0.13825005), (2, 0.09365864), (3, 0.068607055), (4, 0.0379298), (5, 0.17806952), (6, 0.08163874), (7, 0.043054864), (8, 0.10364569), (9, 0.09177349), (10, 0.027877782), (11, 0.033385944), (12, 0.1699042), (13, 0.091722146), (14, 0.049955435), (15, 0.08451286), (16, 0.036626633), (17, 0.03237831), (18, 0.08052498), (19, 0.04803829), (20, 0.033673834), (21, 0.08886209), (22, 0.037412252), (23, 0.028295502), (24, 0.040281795), (25, 0.117281936), (26, 0.07210511), (27, 0.22418098), (28, 0.16818322), (29, 0.03496032), (30, 0.102421865), (31, 0.059233576), (32, 0.10520085), (33, 0.10494021), (34, 0.03748959), (35, 0.11697794), (36, 0.10838953), (37, 0.026465785), (38, 0.06630291), (39, 0.042642903), (40, 0.071987316), (41, 0.0), (42, 0.17453726), (43, 0.0), (44, 0.1622507), (45, 0.0), (46, 0.2652105), (47, 0.0), (48, 0.19594982), (49, 0.14251162), (50, 0.0), (51, 0.09604438), (52, 0.0), (53, 0.03336998), (54, 0.0), (55, 0.031132285), (56, 0.0), (57, 0.1366482), (58, 0.0

In [70]:
sims = sorted(enumerate(sims), key=lambda i: -i[1])

In [71]:
for doc_pos, doc_score in sims[:5]:
    print(doc_score, seg_stop_result[doc_pos])

0.4303464 美國 前 總統 川普痛 批 現任 總統 拜登 砲 火 不 歇 ， 他 說 ， 拜登 讓 美軍 撤離 阿富汗 方式 美國 史上 「 最大 外交政策 恥辱 」 ， 「 這 不是 撤離 ， 這 完全 就是 投降 」 。
0.3799377 她 此行 中 越南 行程 引發 爭議 ， 一些 人 認為 ， 美國 部隊 奮力 要 把 美國 人 、 其他 外國人 阿富汗 盟友 從 喀布爾 機場 撤離 ， 她 卻 選 在 此時 造訪 越南 這個 共產主義 國家 ， 根本就是 聽不進去 別人 聲音 。
0.3199234 根據 民調 ， 半數 受訪者 贊同 拜登 處理 疫情 方式 ， 經濟 政策 支持 度為 39% ， 處理 阿富汗 撤軍 則僅 26% 。 值得注意 ， 民主黨 受訪者 對 拜登 支持 度 仍 高達 87% ， 但 只有 32% 獨立 選民 認為 他 表現 良好 。
0.31380954 民調 顯示 ， 拜登 決定 撤軍 得到 多數 美國 民眾 支持 ， 贊成 反對 比為 53% 對 38% ， 但近 三分之二 受訪者 不 認同 拜登 政府 處理 撤軍 方式 。 參與 民調 獨立 選民 舒倫 布雷 希特 （ Aubrey Schlumbrecht ） 批評 ， 「 拜登 基本上 給了 神 學士 這些 全部 武器 ， 激勵 IS 死灰復燃 ， 他 甚至 不肯 承擔 任何 責任 。 拜登 說 『 責任 歸 他 』 （ he owns ） ， 但是 他 責怪 其他人 ， 他 責怪 阿富汗人 自己 」 。
0.31223425 先前 拋出 2024 年 可能 再選 總統 川普 ， 多次 抨擊 民主 黨籍 拜登 讓 阿富汗 落入 塔利班 （ Taliban ） 手中 ， 即便 引爆 阿富汗 陷落 導火線 美軍 撤離 ， 在 川普 自己 任內 談 成 。


In [72]:
print(sims[:5])

[(180, 0.4303464), (155, 0.3799377), (100, 0.3199234), (102, 0.31380954), (181, 0.31223425)]
